<a href="https://colab.research.google.com/github/luijansuarez/Inventory-Clearance/blob/main/InventoryClearance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://www.savilaeducation.com/" target="_blank">
  <img src="https://res.cloudinary.com/dnq7vo3xt/image/upload/v1752564274/banner_google_collab_white_m4paih.png"
       alt="Company Banner" width="100%">
</a>


# Libraries

In [134]:
'Importamos las librerías que vamos a utilizar'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Import Data

Import sku data

In [135]:
'Leer la data de excel'
df = pd.read_excel('sku_data.xlsx')

In [136]:
'Conocer la "aparencia" del df'
df.head()

,sku,inventory,sales_per_month,price,clearance_price,holding_cost
0,L-FW-8504,873,42,8.93,1.83,0.84
1,N-KI-3619,533,39,7.86,0.50,0.47
2,C-TS-9945,347,4,2.22,0.44,0.05
3,N-QB-3444,848,14,10.85,1.14,0.20
4,W-IF-7282,585,21,6.23,1.05,0.36


In [137]:
'Revisamos cuáles son los tipos de los valores cada columna'
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sku              20000 non-null  object 
 1   inventory        20000 non-null  int64  
 2   sales_per_month  20000 non-null  int64  
 3   price            20000 non-null  float64
 4   clearance_price  20000 non-null  float64
 5   holding_cost     20000 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 937.6+ KB


# Estimate revenue until zero inventory (without discount)

In [138]:
'Determinamos cuál será el rango superior de los ingresos generados con el inventario existente'
df['max_revenue_without_discount'] = df.price * df.inventory

In [139]:
print (F'Max revenue at full price without discount {df['max_revenue_without_discount'].sum():,.2f}')

Max revenue at full price without discount 60,355,233.17


In [140]:
'Determinamos cuál será el rango inferior de los ingresos generados con el inventario existente'
df['max_revenue_clearance_without_discount'] = df.clearance_price * df.inventory

In [141]:
print (F'Max revenue at clearance price without discount {df['max_revenue_clearance_without_discount'].sum():,.2f}')

Max revenue at clearance price without discount 6,127,106.89


In [142]:
'Debido a que es una operación que se debe realizar con cada SKU, creamos una función que corra con todas las variables disponibles'
def calculate_holding_cost_until_zero(row):

  inventory = row['inventory']
  sales = row['sales_per_month']
  holding_cost = row['holding_cost']

  current_inventory = inventory
  total_cost = 0
  while current_inventory > 0:
      total_cost += current_inventory * holding_cost
      current_inventory -= sales
  return total_cost


In [143]:
'Aplicamos la función anterior al df para crear una nueva columna que contenga los costos de almacenamiento hasta que el inventario llegue a cero'
df['holding_cost_until_zero'] = df.apply(calculate_holding_cost_until_zero, axis=1)

In [144]:
'Sumamos los costos de almacenamiento de cada SKU'
print (F'Holding cost until zero {df['holding_cost_until_zero'].sum():,.2f}')

Holding cost until zero 99,790,894.96


In [145]:
'Determinamos la rentabilidad utilizando el rango superior de ingresos menos los costos de almacenamiento'
df['profit_without_discount'] = df['max_revenue_without_discount'] - df['holding_cost_until_zero']

In [146]:
print (F'Profit without discount {df['profit_without_discount'].sum():,.2f}')

Profit without discount -39,435,661.79


In [147]:
'Identificamos el % de SKUs que están generando pérdidas para el negocio'
print(F'% of SKUs that are generating losses {(df['profit_without_discount'] < 0).mean():.2%}')

% of SKUs that are generating losses 28.02%


In [148]:
'Calculamos la rentabilidad si no quisieramos aplicar descuentos si no utilizar el precio de liquidación para aquellos SKUs que están generando perdidas'
profit_profitable = df[df['profit_without_discount']>= 0]['profit_without_discount'].sum()
profit_loss = df[df['profit_without_discount']< 0]['max_revenue_clearance_without_discount'].sum()

print (f'Profit from smart clearance { profit_profitable + profit_loss:,.0f}')

Profit from smart clearance 21,787,454


# Estimate revenue until zero but taking into account the discount value of money

In [149]:
'Generamos una función, muy similar a la anterior, que determinará el costo de almacenamiento tomando en cuenta el valor presente del dinero'
def calculate_cost_until_zero_with_discount(row, annual_rate = 0.05):

    interest_rate = annual_rate/12

    inventory = row['inventory']
    sales = row['sales_per_month']
    holding_cost = row['holding_cost']

    current_inventory = inventory
    total_cost = 0
    t = 0
    while current_inventory > 0:
        total_cost += current_inventory * (holding_cost / ((1+interest_rate)** t))
        current_inventory -= sales
        t += 1
    return total_cost

In [150]:
'Generamos una función que determinará los ingresos tomando en cuenta el valor presente del dinero'
def calculate_revenue_until_zero_with_discount(row, annual_rate = 0.05):

    interest_rate = annual_rate/12

    inventory = row['inventory']
    sales = row['sales_per_month']
    price = row['price']

    current_inventory = inventory
    total_revenue = 0
    t = 0
    while current_inventory > 0:
        total_revenue += sales * (price / ((1 +interest_rate)**t))
        current_inventory -= sales
        t += 1
    return total_revenue

In [151]:
'Calculamos los costos aplicando la tasa de intereses proporcionada para el caso'
df['cost_with_discount'] = df.apply(
    calculate_cost_until_zero_with_discount,
    axis=1,
    annual_rate = 0.04
    )

In [152]:
'Calculamos los costos aplicando la tasa de intereses proporcionada para el caso'
df['revenue_with_discount'] = df.apply(
    calculate_revenue_until_zero_with_discount,
    axis=1,
    annual_rate = 0.04
    )

In [153]:
'Calculamos la rentabilidad tomando en cuenta el Valor Presente del dinero'
df['discount_profit'] = df['revenue_with_discount'] - df['cost_with_discount']

In [154]:
print(df['cost_with_discount'].sum())

80245761.29147047


In [155]:
print(df['revenue_with_discount'].sum())

57012610.41238192


In [156]:
print (F'Profit until zero with discount (4%) {df['discount_profit'].sum():,.0f} $')

Profit until zero with discount (4%) -23,233,151 $


In [157]:
'Calculamos la rentabilidad, tomando en cuenta el Valor Presente del dinero, si no quisieramos aplicar descuentos si no utilizar el precio de liquidación para aquellos SKUs que están generando perdidas'
profit_profitable = df[df['discount_profit']>= 0]['discount_profit'].sum()
profit_loss = df[df['discount_profit']< 0]['max_revenue_clearance_without_discount'].sum()

print (f'Profit from smart clearance { profit_profitable + profit_loss:,.0f}')

Profit from smart clearance 22,172,667


# Determine Discounts

Determine the optimal discount value for each sku, based on marketing fndings

In [158]:
'Creamos un nuevo Data Frame con la información de market_research'
discount_data = pd.read_excel('market_research.xlsx')

In [159]:
discount_data.head()

,product,10,20,30,50,70
0,A,0.00,0.27,0.39,0.71,0.71
1,B,0.45,0.49,0.54,0.54,0.61
2,C,0.04,0.05,0.25,0.25,0.41
3,D,0.03,0.16,0.24,0.26,0.27
4,E,0.32,0.34,0.53,0.59,0.95


In [160]:
'Verificamos si existen valores nulos'
discount_data.isna().sum()

,0
product,0
10,0
20,0
30,0
50,0
70,0


In [161]:
'Revisamos el formato de los valores en este dataframe'
discount_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   product  26 non-null     object 
 1   10       26 non-null     float64
 2   20       26 non-null     float64
 3   30       26 non-null     float64
 4   50       26 non-null     float64
 5   70       26 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.3+ KB


In [162]:
'Vamos a generar un nuevo dataframe independiente cuyos valores sean aquellos SKUs que generan perdidas monetarias después de aplicar el valor presente del dinero'
lossing_product = df[df['discount_profit'] < 0].iloc[:,:6].reset_index(drop=True).copy()

In [163]:
lossing_product

,sku,inventory,sales_per_month,price,clearance_price,holding_cost
0,L-FW-8504,873,42,8.93,1.83,0.84
1,C-TS-9945,347,4,2.22,0.44,0.05
2,W-CN-7224,654,8,4.93,0.98,0.30
3,J-IU-6818,742,22,6.51,0.53,0.47
4,X-JV-9924,918,20,7.77,-0.19,0.46
...,...,...,...,...,...,...
5605,O-FW-7476,691,21,2.07,0.40,0.18
5606,J-ZD-1041,117,2,5.95,0.13,0.48
5607,H-AM-9158,712,7,3.36,0.06,0.20
5608,B-JM-4516,413,7,7.77,1.10,0.39


In [164]:
'Creamos una función que determine la rentabilidad tomando en cuenta el valor presente del dinero. A diferencia de anteriormente, en esta función se calcula los ingresos y costos en la misma función.'
def profit_discount (inventory, sales_per_month, price, holding_cost, annual_rate = 0.05):

    interest_rate = annual_rate/12

    current_inventory = inventory
    total_profit = 0
    t = 0
    while current_inventory > 0:
      sold = min(current_inventory, sales_per_month)
      revenue = sold * (price / ((1 +interest_rate)**t))
      costs = current_inventory * (holding_cost / ((1+interest_rate)** t))
      total_profit += (revenue - costs)
      current_inventory -= sold
      t += 1
    return total_profit

In [165]:
'Vamos a crear un Dictionary en el cual se tomará aquellos SKUs que están generando perdidas y se determinará si se deben vender con precio de liquidación o con un descuento. Es importante mencionar que no se está buscando una rentabilidad positiva para todos los SKUs, simplemente generar la menor perdida posible.'
decision_losing_products = dict()

price_discount = list(discount_data.columns[1:])

for idx, row in lossing_product.iterrows():

    sku,	inventory,	sales_per_month,	price,	clearance_price,	holding_cost = row
    product_category = sku.split('-')[0]
    lift = discount_data[discount_data['product'] == product_category].values[0][1:]

    clearance_revenue = inventory * clearance_price
    decision = 'clearance'
    best_profit = clearance_revenue

    for s, p in zip(lift, price_discount):
      lift_sales = sales_per_month * (1+s)
      price_disc = price * (1-(p/100))
      profit = profit_discount(inventory, lift_sales, price_disc, holding_cost, 0.04)
      if profit > best_profit:
        decision = f'{p}% discount'
        best_profit = profit
    decision_losing_products[sku] = (decision, best_profit)



In [166]:
decision_losing_products

{'L-FW-8504': ('clearance', 1597.5900000000001),
 'C-TS-9945': ('clearance', 152.68),
 'W-CN-7224': ('clearance', 640.92),
 'J-IU-6818': ('clearance', 393.26000000000005),
 'X-JV-9924': ('clearance', -174.42000000000002),
 'G-YO-8975': ('clearance', 521.09),
 'R-BB-9827': ('clearance', 1059.03),
 'Q-IU-9754': ('clearance', 322.83),
 'V-LA-8293': ('clearance', 74.5),
 'H-PE-6622': ('clearance', -31.450000000000003),
 'E-UB-4506': ('clearance', 367.12),
 'T-BT-2743': ('clearance', -74.72),
 'N-VF-5257': ('clearance', 68.60000000000001),
 'N-TZ-5915': ('clearance', 136.79999999999998),
 'M-DX-6158': ('clearance', -87.21000000000001),
 'K-MW-8242': ('clearance', 66.0),
 'Q-GF-2144': ('clearance', 1435.0),
 'C-XV-3734': ('clearance', 1228.77),
 'N-EM-2078': ('clearance', 1192.98),
 'I-AV-7179': ('clearance', 102.89999999999999),
 'R-VY-3828': ('clearance', 858.52),
 'P-QC-2709': ('clearance', 204.24),
 'H-UV-6930': ('10% discount', 402.53576069686864),
 'I-QW-4940': ('clearance', 765.06),
 

In [167]:
'Con el uso del dictionary anterior ahora podemos asociar el resultado de ese análisis a nuestro data frame lossing_product, en este caso extraemos la decisión que se tomo'
lossing_product['decision'] = lossing_product['sku'].map(lambda x: decision_losing_products[x][0])

In [168]:
lossing_product['decision'].value_counts()

,count
decision,
clearance,5473
10% discount,81
30% discount,38
20% discount,18


In [169]:
'Con el uso del dictionary anterior ahora podemos asociar el resultado de ese análisis a nuestro data frame lossing_product, en este caso extraemos la rentabilidad que se calculó'
lossing_product['profit'] = lossing_product['sku'].map(lambda x: decision_losing_products[x][1])

In [170]:
lossing_product['profit'].describe()

,profit
count,5610.000000
mean,414.591557
std,477.501740
min,-496.260000
25%,72.390000
50%,278.975000
75%,639.772500
max,2552.820000


# Generate the reports

In [171]:
'Creamos un nuevo data frame'
out_df = pd.DataFrame()

In [172]:
'Incluimos la columna de sku de nuestro df al data frame creado.'
out_df['sku'] = df['sku']

In [173]:
out_df.head()

,sku
0,L-FW-8504
1,N-KI-3619
2,C-TS-9945
3,N-QB-3444
4,W-IF-7282


In [174]:
'Esta función separa el SKU por el guión y nos devuelve el tipo de producto'
def product_category(row):
  return row['sku'].split('-')[0]

In [175]:
'Creamos la columna de categoría de producto'
out_df['product_category'] = out_df.apply(product_category, axis=1)

In [176]:
'para todos aquellos SKUs que no tienen valor de decision ya que eran rentables desde el principio, se les asigna el valor de ok'
from textwrap import fill
out_df['decision'] = out_df['sku'].map(lambda x: decision_losing_products.get(x,(np.nan,np.nan))[0]).fillna('ok')

In [177]:
'Copiamos la rentabilidad calculada según descuento/clearance'
out_df['final_profit'] = out_df['sku'].map(lambda x: decision_losing_products.get(x,(np.nan,np.nan))[1])

In [178]:
'Para los SKUs rentables copiamos la rentabilidad inicial con su precio de venta full.'
out_df['final_profit'][out_df['final_profit'].isna()] = df['discount_profit']

/tmp/ipython-input-3130191619.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  out_df['final_profit'][out_df['final_profit'].isna()] = df['discount_profit']
/tmp/ipython-input-3130191619.py:2: SettingWithCopyWarning: 
A value is trying to

In [179]:
out_df['final_profit']

,final_profit
0,1597.590000
1,2386.606600
2,152.680000
3,3506.705326
4,552.963298
...,...
19995,1066.402463
19996,659.511055
19997,3086.934841
19998,360.948764


In [180]:
print(f'Best escenario profit {out_df['final_profit'].sum():,.0f} $')

Best escenario profit 22,204,381 $


# Create columns for report

## Excel notebook

In [181]:
'Nos aseguramos de tener instalada una librería para poder manipular excel con pandas'
! pip install openpyxl

In [182]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill

In [183]:
'Determinamos de qué color serán rellenas las celdas según la estrategia de ventas de acuerdo a las instrucciones dadas'
fill_clear = PatternFill(start_color='FFC9C9',
                   end_color='FFC9C9',
                   fill_type='solid')
fill_discount = PatternFill(start_color='FEF9C2',
                   end_color='FEF9C2',
                   fill_type='solid')

In [184]:
'Creamos el archivo de excel. Es importante recordar que Excel empieza a contar desde el 1 y en el caso de filas la 1 siempre es el header por lo que los valores se empiezan a almacenar en la 2.'
output_path = 'Inventory_clearance_report.xlsx'
writer = pd.ExcelWriter(output_path, engine='openpyxl')

for p_type in sorted (out_df['product_category'].unique()):
  temp = out_df[out_df['product_category'] == p_type].copy()

  temp.to_excel(writer, sheet_name= f'{p_type}', index=False)

  sheet = writer.book[f'{p_type}']

  for row_idx in range(2,len(temp)+2):

    decision = sheet[f"{chr(65 + temp.columns.get_loc('decision'))}{row_idx}"].value

    if decision == 'clearance':
      for col_idx in range(1,len(temp.columns)+1):
        sheet.cell(row = row_idx, column = col_idx).fill = fill_clear

    elif "discount" in decision:
      for col_idx in range(1,len(temp.columns)+1):
        sheet.cell(row = row_idx, column = col_idx).fill = fill_discount

writer.close()

## Word summary report

In [185]:
'Nos aseguramos de tener una librería que pueda manipular word.'
!pip install python-docx

In [186]:
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

In [187]:
'Creamos un tabla con la información necesaria para el resumen ejecutivo'
pivot = (
    pd.pivot_table(
        out_df,
        index='product_category',
        columns='decision',
        values='sku',
        aggfunc='count',
        fill_value=0
    )
    .reset_index()
)
pivot['discount'] = pivot['10% discount'] + pivot['20% discount'] + pivot['30% discount']

In [188]:
pivot.columns

Index(['product_category', '10% discount', '20% discount', '30% discount',
       'clearance', 'ok', 'discount'],
      dtype='object', name='decision')

In [189]:
pivot

decision,product_category,10% discount,20% discount,30% discount,clearance,ok,discount
0,A,0,4,0,221,549,4
1,B,20,0,0,194,541,20
2,C,0,0,0,210,569,0
3,D,0,0,0,220,521,0
4,E,8,0,0,219,552,8
5,F,17,0,0,218,559,17
6,G,0,0,1,200,547,1
7,H,16,0,0,192,564,16
8,I,5,0,0,219,500,5
9,J,0,0,5,226,592,5


In [190]:
pivot = pivot [['product_category', 'discount','clearance','ok']]

In [191]:
pivot

decision,product_category,discount,clearance,ok
0,A,4,221,549
1,B,20,194,541
2,C,0,210,569
3,D,0,220,521
4,E,8,219,552
5,F,17,218,559
6,G,1,200,547
7,H,16,192,564
8,I,5,219,500
9,J,5,226,592


In [192]:
'Lógica para la creación del archivo de texto.'
doc = Document()
logo_path = 'company_logo.png'

try:
  logo_paragraph = doc.add_paragraph()
  logo_run = logo_paragraph.add_run()
  logo_run.add_picture(logo_path, width=Inches(1.5))
  logo_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
except Exception as e:
  print(f'Error adding logo:', e)

doc.add_heading("Inventory Clearance Summary Report", level=1)

total_clear = (out_df['decision'] == 'clearance').sum()
total_discount = total_discount = (out_df['decision'].str.contains('discount')).sum()

total_profit = out_df['final_profit'].sum()

executive_text= f"""
This report summarizes the inventory clearance recomendations for all product types.

A total of {total_clear} SKUs require clearance, indicating inmediate action is necessary to reduce holding costs. A total of {total_discount} can be sold at a discount to optimize profitability.

The remaining SKUs fall into discount or no-action categories, allowing a more strategic approach to inventory optimization. By following this strategy the company can get a total profit of its current inventory of $ {total_profit:,.0f}.
"""
for block in executive_text.strip().split("\n\n"):
    p= doc.add_paragraph(block)
    p.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
doc.add_heading("SKU Decision Counts per Product Type", level = 2)

rows, cols = pivot.shape
table = doc.add_table(rows=rows+1, cols=cols)
table.style = "Light List"

for j, col_name in enumerate(pivot.columns):
    table.rows[0].cells[j].text =  str(col_name)
    table.rows[0].cells[j].paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    for i in range(rows):
      for j in range(cols):
        table.rows[i+1].cells[j].text = str(pivot.iloc[i,j])
        table.rows[i+1].cells[j].paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
output_path = 'Inventory_clearance_summary.docx'
doc.save(output_path)

print("Word report created", output_path)


Word report created Inventory_clearance_summary.docx
